# Exploring Metaflow
> Want to use this to take a look at how metaflow works, specifically their decorators

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]
- image: images/chart-preview.png

## About 
Metaflow is a python package open sourced by netflix to help data scientists easily scale their project workflows. Metaflow is mainly interacted with through decorators. In this post, I want to get behind the scenes of how these decorators actually work. 

In [ ]:
#collapse-show

from metaflow import FlowSpec, step
class LinearFlow(FlowSpec):
    @step
    def start(self):
        self.my_var = 'hello world'
        self.next(self.a)

    @step
    def a(self):
        print('the data artifact is: %s' % self.my_var)
        self.next(self.end)

    @step
    def end(self):
        print('the data artifact is still: %s' % self.my_var)

LinearFlow()